In [2]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states
from config import g_key

# Census API Key
from config import census_key

c = Census(census_key, year=2018)
# print(census_key)


ModuleNotFoundError: No module named 'us'

In [3]:
# Run Census Search to retrieve data on all zip codes 
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E", "B01002_002E",
                          "B01002_003E","B19301_001E"),
                           {'for': 'zip code tabulation area:*'})
                         
census_data
# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B01002_002E": "Median Male Age",
                                      "B01002_003E": "Median Female Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",                                      
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})
census_pd                   


NameError: name 'c' is not defined

In [4]:

# # Add in Per Capita Income Rate (Per Capita Income / Population)
# census_pd["Total Income"] = 100 * \
#     census_pd["Per Capita Income"].astype(
#         int) / census_pd["Population"].astype(int)

# # Final DataFrame
# census_pd = census_pd[["State", "Population", "Median Age", "Median Male Age", "Median Female Age", "Household Income",
#                        "Per Capita Income Rate", "Per Capita Income"]]

# # Visualize
# print(len(census_pd))
# census_pd.head()

In [18]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
census_pd.to_csv("census_data_states.csv", encoding="utf-8", index=False)

In [25]:
uszips = pd.read_csv("uszips.csv")
uszips.head()

,Zip,lat,lng,city,state_id,State
0,601,18.18004,-66.75218,Adjuntas,PR,Puerto Rico
1,602,18.36073,-67.17517,Aguada,PR,Puerto Rico
2,603,18.45439,-67.12202,Aguadilla,PR,Puerto Rico
3,606,18.16724,-66.93828,Maricao,PR,Puerto Rico
4,610,18.29032,-67.12243,Anasco,PR,Puerto Rico


In [26]:
# Merge the datasets using the sate columns
census_data = pd.merge(census_pd, uszips, how="left", left_on="Name", right_on="State" )

# Save the updated dataframe as a csv
census_data.to_csv("state_census_data.csv", encoding="utf-8", index=False)
census_data.head()

,Name,Household Income,Population,Median Age,Median Male Age,Median Female Age,Per Capita Income,Zipcode,Zip,lat,lng,city,state_id,State
0,ZCTA5 00601,13092.0,17242.0,40.5,39.5,41.3,6999.0,00601,NaN,NaN,NaN,NaN,NaN,NaN
1,ZCTA5 00602,16358.0,38442.0,42.3,41.9,42.8,9277.0,00602,NaN,NaN,NaN,NaN,NaN,NaN
2,ZCTA5 00603,16603.0,48814.0,41.1,39.1,42.9,11307.0,00603,NaN,NaN,NaN,NaN,NaN,NaN
3,ZCTA5 00606,12832.0,6437.0,43.3,43.2,43.4,5943.0,00606,NaN,NaN,NaN,NaN,NaN,NaN
4,ZCTA5 00610,19309.0,27073.0,42.1,40.4,44.2,10220.0,00610,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# # Convert poverty rate as a list
# # Convert watch rate to list
# poverty_rate = census_data["Poverty Rate"].tolist()

In [ ]:
# Configure gmaps
gmaps.configure(api_key = g_key)

In [ ]:
# # Create a map using state centroid coordinates to set markers
# marker_locations = census_data[['Latitude', 'Longitude']]

# # Create a marker_layer using the poverty list to fill the info box
# fig = gmaps.figure()
# markers = gmaps.marker_layer(marker_locations,
#     info_box_content=[f"Poverty Rate: {rate}" for rate in poverty_rate])
# fig.add_layer(markers)
# fig